In [1]:
from src import angles as ang
from fractions import Fraction as Frac
# import numpy as np
# from numpy import sort, array, where, append
from numpy import sort as npsort
from numpy import array as nparray
from numpy import ones as npones
from numpy import where as npwhere
from numpy import append as npappend

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs

from numpy.linalg import eigvals as nplinalgeigvals

In [2]:
def coreEntropy(num, den):
    #define a new rational angle
    theta = ang.Angle(num,den)
    
    thetaFr = Frac(theta.num,theta.den)#represent it as a fraction
   
    #compute the orbit and find the period
    theta.period()
    orb = theta.orbit_list
    period_length = theta.per_len
    period_start = theta.start_index_per
    preperiod_length = len(orb)-1-period_length
    
    #partition of the circle
    intOne = (thetaFr*Frac(1,2),(thetaFr+Frac(1,1))*Frac(1,2))
    
    print(f"The angle is {thetaFr} \n which has a preperiod of {preperiod_length} and a period of {period_length} \n and the orbit is {orb}\n")
    print(f"partition the circle in two intervals: {intOne} and {(intOne[1],intOne[0])}\n")
    
    #create the vertex set of the wedge
    tuples = nparray(["%d-%d"%(i,j) for i in range(1,len(orb)) for j in range(1,len(orb)) if i<j])
    print(f"tuples is {tuples}\n")
    
    #define the separated and non-separated vertices
    dicTuples = {}
    entries = 0
    indices = nparray([],dtype=int)
    indptr = nparray([0],dtype=int)
    
    for tup in tuples:
        i,j=tup.split('-')
        i=int(i)
        j=int(j)
        max_ind = len(orb)-1
        
        print(f"the tuple {tup} corresponding to the angles {orb[i-1]} and {orb[j-1]}")

        if ((intOne[0]<orb[i-1]<=intOne[1] and intOne[0]<orb[j-1]<=intOne[1]) or (intOne[1]<=orb[i-1] and intOne[1]<=orb[j-1])):
            print("is not separated")
            if (j+1)<=max_ind:
                target = str(i+1)+'-'+str(j+1)
            else:
                new_j = (j+1+preperiod_length)%max_ind
                new_i = i+1
                target = str(new_i)+'-'+str(new_j) if new_i<new_j else str(new_j)+'-'+str(new_i)
            print(f"target = {target}")
            index = npwhere(tuples==target)[0][0]
            print(f"the target is {target} which has index {index}\n")
            indices = npappend(indices,index)
            dicTuples.update({tup:{'sep':False,'mapsTo':[target]}})
            entries += 1
        else:
            print("is separated")
            target_one = str(1)+'-'+str(i+1) if (i+1)<=max_ind else str(1)+'-'+str((i+1+preperiod_length)%max_ind)
            print(f"target one = {target_one}")
            target_two = str(1)+'-'+str(j+1) if (j+1)<=max_ind else str(1)+'-'+str((j+1+preperiod_length)%max_ind)
            print(f"target two = {target_two}")
            index_one = npwhere(tuples==target_one)[0][0]
            index_two = npwhere(tuples==target_two)[0][0]
            print(f"the target_one is {target_one} which has index {index_one}")
            print(f"the target_two is {target_two} which has index {index_two}\n")
            indices = npappend(indices,index_one)
            indices = npappend(indices,index_two)
            dicTuples.update({tup:{'sep':True,'mapsTo':[target_one,target_two]}})
            entries += 2
        indptr = npappend(indptr,entries)
    
    data = npones(entries,dtype='float64')
    adj_matrix = csr_matrix((data, indices, indptr), shape=(len(tuples),len(tuples)))
    evals = eigs(adj_matrix,k=len(tuples)-2, which='LM',sigma=0.999,return_eigenvectors=False)
    
    print(dicTuples)
    print("\n")
    print(f"data = {data}")
    print(f"indices = {indices}")
    print(f"indptr = {indptr}")
    print(csr_matrix((data, indices, indptr), shape=(len(tuples),len(tuples))).toarray())
    print(f"evals using sparse {evals}")
    M = adj_matrix.toarray()
    evals_two = nplinalgeigvals(M)
    print(f"evals using numpy {evals_two}")
    
            
    

In [3]:
coreEntropy(329,960)

The angle is 329/960 
 which has a preperiod of 6 and a period of 4 
 and the orbit is [Fraction(329, 960), Fraction(329, 480), Fraction(89, 240), Fraction(89, 120), Fraction(29, 60), Fraction(29, 30), Fraction(14, 15), Fraction(13, 15), Fraction(11, 15), Fraction(7, 15), Fraction(14, 15)]

partition the circle in two intervals: (Fraction(329, 1920), Fraction(1289, 1920)) and (Fraction(1289, 1920), Fraction(329, 1920))

tuples is ['1-2' '1-3' '1-4' '1-5' '1-6' '1-7' '1-8' '1-9' '1-10' '2-3' '2-4' '2-5'
 '2-6' '2-7' '2-8' '2-9' '2-10' '3-4' '3-5' '3-6' '3-7' '3-8' '3-9' '3-10'
 '4-5' '4-6' '4-7' '4-8' '4-9' '4-10' '5-6' '5-7' '5-8' '5-9' '5-10' '6-7'
 '6-8' '6-9' '6-10' '7-8' '7-9' '7-10' '8-9' '8-10' '9-10']

the tuple 1-2 corresponding to the angles 329/960 and 329/480
is separated
target one = 1-2
target two = 1-3
the target_one is 1-2 which has index 0
the target_two is 1-3 which has index 1

the tuple 1-3 corresponding to the angles 329/960 and 89/240
is not separated
target = 2-4


In [14]:

{
    '1-2': {'sep': False, 'mapsTo': ['2-3']}, 
    '1-3': {'sep': True, 'mapsTo': ['1-2', '1-4']}, 
    '1-4': {'sep': False, 'mapsTo': ['1-2']}, 
    '2-3': {'sep': True, 'mapsTo': ['1-3', '1-4']}, 
    '2-4': {'sep': False, 'mapsTo': ['1-3']}, 
    '3-4': {'sep': False, 'mapsTo': ['1-4']}}
{
    (1, 2): {'sep': False, 'mapsTo': [(2, 3)]},
    (1, 3): {'sep': True, 'mapsTo': [(1, 2), (1, 4)]}, 
    (1, 4): {'sep': False, 'mapsTo': [(1, 2)]}, 
    (2, 3): {'sep': True, 'mapsTo': [(1, 3), (1, 4)]}, 
    (2, 4): {'sep': False, 'mapsTo': [(1, 3)]}, 
    (3, 4): {'sep': False, 'mapsTo': [(1, 4)]}
}
{
    (1, 2): {'sep': False, 'mapsTo': [(2, 3)]},
    (1, 3): {'sep': False, 'mapsTo': [(1, 2)]},
    (2, 3): {'sep': False, 'mapsTo': [(1, 3)]}
}

{(1, 2): {'sep': False, 'mapsTo': [(2, 3)]},
 (1, 3): {'sep': False, 'mapsTo': [(1, 2)]},
 (2, 3): {'sep': False, 'mapsTo': [(1, 3)]}}

In [2]:
from src import functions as fcn

In [4]:
sparseMat = fcn.Sparse(3,3)
sparseMat.insert(0,0,1)
sparseMat.insert(1,1,1)
sparseMat.insert(1,2,1)
sparseMat.insert(2,1,1)
sparseMat.insert(0,3,1)
print(sparseMat)

Shape: 3 x 3
[0, 0, 1]
[0, 3, 1]
[1, 1, 1]
[1, 2, 1]
[2, 1, 1]
Total: 5


In [34]:
a,b="19-23".split('-')
print(f"a={a} and b={b}")
print(f"type {type(a)}")

a=19 and b=23
type <class 'str'>


In [2]:
from src import functions as fcn
from src import angles

angles.Angle(1,5)
print(angles.Angle(2,7))
# fcn.core_entropy(1,5)

The angle is 2/7
